In [24]:
%load_ext autoreload
%autoreload 2
from preprocess import *
from displays import *

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

raw_main_df = pd.read_csv('data/train.csv')
raw_sub_df = pd.read_csv('data/test.csv')

raw_train_df, raw_test_df = train_test_split(raw_main_df, test_size=100, random_state=42)

raw_train_df.sample(5, random_state=42)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
678,679,0,3,"Goodwin, Mrs. Frederick (Augusta Tyler)",female,43.0,1,6,CA 2144,46.9000,NaN,S
49,50,0,3,"Arnold-Franchi, Mrs. Josef (Josefine Franchi)",female,18.0,1,0,349237,17.8000,NaN,S
691,692,1,3,"Karun, Miss. Manca",female,4.0,0,1,349256,13.4167,NaN,C
499,500,0,3,"Svensson, Mr. Olof",male,24.0,0,0,350035,7.7958,NaN,S
303,304,1,2,"Keane, Miss. Nora A",female,NaN,0,0,226593,12.3500,E101,Q


# Funs

In [2]:
from utils import displays
from utils2 import displays

#ut.preprocess.f(2)

-----
# Highest Freq estimator

In [13]:
freq_1 = np.sum(raw_train_df.Survived) / raw_train_df.shape[0]
freq_0 = 1 - freq_1
print(freq_0, freq_1)

0.618204804045512 0.38179519595448796


In [ ]:
def highest_freq_estimator(X):
    return 0



------
# 2. Logistic Regression
## 2.1. Prepare Train, Test sets

In [5]:
from sklearn.linear_model import LogisticRegression

PARAMS = {
}
COLS_IGNORED = ['PassengerId', 'Name', 'Ticket']

train_df = (
    raw_train_df
    .copy()
    .pipe(remove_unusable_cols, COLS_IGNORED)
    .pipe(keep_first_letter)
    .pipe(replace_cols_nans, raw_main_df, show_nans=False)
    .pipe(remove_nans)
    .pipe(remove_outliers)
    .pipe(convert_types)
    .pipe(get_dummies)
)

X_train, y_train = train_df.drop('Survived', axis=1), train_df.Survived

#.fit(X_train, y_train)

> Outliers: 24 / 791


## 2.2. Cross Validation

In [6]:
import random
C_par = 10*10**(-np.random.uniform(0,2, size=10))
C_par

array([5.18606758, 0.16774251, 6.13860292, 2.3663118 , 0.89376953,
       0.13394837, 0.55373462, 1.69005142, 0.19399742, 7.21911308])

In [7]:
import warnings
warnings.filterwarnings('ignore')

# Best: 0.186015041630706

for c in C_par:
    print('C :', c)
    model = LogisticRegression(max_iter=100, random_state=0, C=c)
    val_scores = cross_val_score(
        model,
        X_train,
        y_train,
        cv=10,
        scoring= lambda esti, X, y_true: roc_auc_score(y_true, esti.predict_proba(X)[:,1])
    )
    print(np.mean(val_scores))#, val_scores)
    print('-----')



#np.mean(val_scores)
# 500 -> 0.8453696546483597
# 100 -> 0.846616466652692
#0.23, 0.021472848189418532, 0.77, 0.47

C : 5.186067583606398


NameError: name 'cross_val_score' is not defined

In [9]:
BEST_PARAMS = {
    'C': 0.186015041630706
}
final_model = LogisticRegression(max_iter=500, random_state=0, C=BEST_PARAMS['C']).fit(X_train, y_train)

## 2.2. Evaluate model

In [23]:
SUBM_PATH = f'submissions/log_{PARAMS}.csv'
COLS_IGNORED = ['Name', 'Ticket']

subm_features_df = (
    raw_sub_df
    .copy()
    .drop(COLS_IGNORED, axis=1)
    .pipe(keep_first_letter)
    .pipe(replace_cols_nans, ref_df=raw_train_df, show_nans=False)
    .pipe(convert_types, submit=True)
    .pipe(get_dummies)
    .pipe(set_same_dummies_as_ref, train_df)
)

X_subm = subm_df.drop('PassengerId', axis=1)
y_pred = model.predict(X_subm)

subm_df = pd.DataFrame({
    'PassengerId': subm_df.PassengerId,
    'Survived': y_pred
})

subm_df.to_csv(SUBM_PATH, index=False)